In [2]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("data.csv")

df = df[df["CONSENT"] == "Yes, I accept"]

right_answers = {
    "photo1": "Real photo",
    "photo2": "AI generated",
    "photo3": "AI generated",
    "photo4": "AI generated",
    "photo5": "Left Picture",
    "photo6": "Right Picture",
    "photo7": "Right Picture"
}

#NEW COLUMN

df["RIGHT ANSWERS"]= df[list(right_answers.keys())].eq(pd.Series(right_answers)).sum(axis=1)
database = df[["AGE","SOCIAL MEDIA USAGE","MAIN SOCIAL MEDIA","CONFIDENCE","RIGHT ANSWERS"]]

database.head()



,AGE,SOCIAL MEDIA USAGE,MAIN SOCIAL MEDIA,CONFIDENCE,RIGHT ANSWERS
0,20,1-3 hours,Instagram,4,5
1,20,3-5 hours,TikTok,4,6
2,20,1-3 hours,TikTok,3,5
3,18,1-3 hours,TikTok,5,5
4,21,3-5 hours,Instagram,3,6


In [ ]:
# overall accuracy vs age, histogram
# not sure if we acutally need bootstrapping since the age in continuous, but check it

# please use the following databse variable in this part, so u do not interfere with the other parts
df_1 = database.copy()


In [ ]:
# overall accuracy vs social med usage, histogram
# use bootstrapping bcs the groups are uneven

# please use the following database variable in this part, so u do not interfere with the other parts
df_2 = database.copy()


In [ ]:
# generalized linear model, (age and usage) vs right answers column

# please use the following database variable in this part, so u do not interfere with the other parts
df_3 = database.copy()
